In [2]:
import pandas as pd
import hashlib
import numpy as np

In [4]:
df = pd.read_csv("../../data/trusted/arquivo_anonimizado_v2.csv")

In [64]:
def processar_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)

In [65]:
def pegar_disciplinas(x):
    if x[0] == 'nan':
        return np.nan
    try:
        return (
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]),
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [66]:
# Converte a coluna 'disciplinasCursadas' para string
df["disciplinasCursadas"] = df["disciplinasCursadas"].astype(str)

# Processa a coluna 'disciplinasCursadas' para separar em periodo, disciplina, grau, stituação
df["disciplinasCursadas"] = df["disciplinasCursadas"].apply(lambda row: [processar_disciplina(item) for item in row.split("\n")])

# Explode a coluna 'disciplinasCursadas' para dividir as listas em linhas separadas
df_exploded = df.explode("disciplinasCursadas")

df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(pegar_disciplinas)

df_exploded.dropna(inplace=True)

In [67]:
df_disciplina = pd.DataFrame(
    df_exploded["disciplinasCursadas"].tolist(),
    index=df_exploded.index,
    columns=(
        "cod_disciplina",
        "disciplina",
    ),
)

d_disciplina = df_disciplina.drop_duplicates().reset_index(drop=True)

In [68]:
d_disciplina["sk_d_disciplina"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["cod_disciplina"] + x["disciplina"]).encode()
    ).hexdigest(),
    axis=1,
)

colunas_disciplina = ["sk_d_disciplina", "cod_disciplina", "disciplina"]

d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)

In [69]:
d_disciplina.to_csv("../../data/refined/d_disciplina.csv", index=False)